In [3]:
import requests 
from bs4 import BeautifulSoup 
from time import sleep 
import pandas as pd 

In [5]:
df = pd.read_csv('asunews.csv')

In [69]:
keywords = ['international', 'AI', 'Homecoming']                    

In [40]:
url = 'https://news.asu.edu/' 
homepage = requests.get(url)
bs = BeautifulSoup(homepage.text,'html.parser')

In [52]:
article_links = []

for a_tag in bs.select('div.card-title a, div.card-header a'):
    href = a_tag.get('href')
    if href:
        full_url = href if href.startswith('http') else 'https://news.asu.edu' + href
        article_links.append(full_url)

In [70]:
def grabarticle(thisurl):
    thispage = requests.get(thisurl)
    bs = BeautifulSoup(thispage.text,'html.parser')

    # Try both possible article containers
    body = bs.find(class_='card-body') or bs.find(class_='story-body') or bs.find(class_='view-content')
    if not body:
        return None  

    # Extract paragraphs safely
    paragraphs = body.find_all('p')
    article_text = '\n\n'.join(p.text for p in paragraphs)

    # Title
    title = bs.find('h1').text if bs.find('h1') else 'No Title'

    # Author
    author_tag = bs.find(class_='author-bio')
    author = author_tag.text.strip() if author_tag else 'No Author'

    # Date
    dateline_tag = bs.find(class_='small')
    dateline = dateline_tag.text.strip() if dateline_tag else 'No Date'

    return {
        'url': thisurl,
        'author': author,
        'title': title,
        'dateline': dateline,
        'article': article_text   # <-- returns TEXT, not a tag
    }

In [72]:
for eachlink in article_links:
    if not df.url.str.contains(eachlink).any():
        print('getting', eachlink)
        story = grabarticle(eachlink)
        sleep(5)
        if story:
            for kw in keywords:
                if kw in story['article'].lower():
                    story['keyword'] = kw
                    df = pd.concat([df, pd.DataFrame([story])], ignore_index=True)
                    break

getting https://news.asu.edu/20251120-university-news-asu-launches-asu-london-bringing-global-reputation-innovation-uk
getting https://news.asu.edu/20251117-sun-devil-community-asu-remains-no-1-public-university-choice-international-students
getting https://news.asu.edu/20251031-environment-and-sustainability-asu-program-educates-real-estate-professionals-arizona-water-issues
getting https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability
getting https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead
getting https://news.asu.edu/20250915-science-and-technology-asu-student-works-protect-african-penguins-local-aquarium
getting https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix
getting https://news.asu.edu/20251118-science-and-technology-collaboration-key-success-asus-forensic-school-director-says
getting https://news.asu.edu/20251114-la

In [73]:
df.to_csv('asunews.csv')

In [74]:
df

,url,author,title,dateline,article,keyword
0,https://news.asu.edu/20251119-sun-devil-commun...,"By Marshall Terrill | \n\nNovember 19, 2025",ASU launches new training to strengthen suppor...,"November 19, 2025","For the fifth consecutive year, Arizona State ...",international
